In [1]:
#importing libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from matplotlib import pyplot
from scipy import stats
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [3]:
#Reading excel for ARIMA
df=pd.read_excel("Forecasting Assignment Data.xlsx","Sheet1",index_col=[0],parse_dates=True,squeeze=True)

In [4]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [5]:
a=df.filter(['Maharashtra'], axis=1)
# define input sequence
timeseries_data = np.array(a)
# choose a number of time steps
n_steps = 3
# split into samples
X, y = prepare_data(timeseries_data, n_steps)

In [6]:
X

array([[[428.6],
        [419.6],
        [395.8]],

       [[419.6],
        [395.8],
        [411.1]],

       [[395.8],
        [411.1],
        [408.6]],

       ...,

       [[485.9],
        [494.1],
        [492.8]],

       [[494.1],
        [492.8],
        [472.8]],

       [[492.8],
        [472.8],
        [475.3]]])

In [7]:
X.shape

(500, 3, 1)

In [8]:

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [9]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
16/16 [==============================] - 6s 18ms/step - loss: 128106.7344
Epoch 2/50
16/16 [==============================] - 0s 12ms/step - loss: 48966.9297
Epoch 3/50
16/16 [==============================] - 0s 16ms/step - loss: 3980.8218
Epoch 4/50
16/16 [==============================] - 0s 18ms/step - loss: 1224.9657
Epoch 5/50
16/16 [==============================] - 0s 16ms/step - loss: 1059.5623
Epoch 6/50
16/16 [==============================] - 0s 15ms/step - loss: 976.8330
Epoch 7/50
16/16 [==============================] - 0s 12ms/step - loss: 960.0760
Epoch 8/50
16/16 [==============================] - 0s 14ms/step - loss: 898.7351
Epoch 9/50
16/16 [==============================] - 0s 14ms/step - loss: 826.4261
Epoch 10/50
16/16 [==============================] - 0s 10ms/step - loss: 768.8766
Epoch 11/50
16/16 [==============================] - 0s 18ms/step - loss: 776.1995
Epoch 12/50
16/16 [==============================] - 0s 18ms/step - loss: 733.4136
Epoch

In [10]:
from array import array

In [11]:
y_input = np.array(a[-3:])

In [12]:
a[-3:]

Maharashtra
Dates                  
2020-03-12        472.8
2020-04-12        475.3
2020-05-12        470.5

In [20]:
# demonstrate prediction for next 10 days
x_input = np.array([132.1,132.1,120.4])
temp_input=list(x_input)
lst_output=[]
i=0
while(i<90):
    
    if(len(temp_input)>3):
        x_input=np.array(temp_input[1:])
        print("{} month input {}".format(i,x_input))
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} month output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    

print(lst_output)

[147.53262]
1 month input [132.1        120.4        147.53262329]
1 month output [[154.98875]]
2 month input [120.4        147.53262329 154.98875427]
2 month output [[153.60063]]
3 month input [147.53262 154.98875 153.60063]
3 month output [[172.84213]]
4 month input [154.98875 153.60063 172.84213]
4 month output [[185.55624]]
5 month input [153.60063 172.84213 185.55624]
5 month output [[191.73259]]
6 month input [172.84213 185.55624 191.73259]
6 month output [[207.47083]]
7 month input [185.55624 191.73259 207.47083]
7 month output [[222.88428]]
8 month input [191.73259 207.47083 222.88428]
8 month output [[235.09763]]
9 month input [207.47083 222.88428 235.09763]
9 month output [[249.6918]]
10 month input [222.88428 235.09763 249.6918 ]
10 month output [[265.1]]
11 month input [235.09763 249.6918  265.1    ]
11 month output [[279.5248]]
12 month input [249.6918 265.1    279.5248]
12 month output [[293.53433]]
13 month input [265.1     279.5248  293.53433]
13 month output [[306.9820

In [14]:
day_new=np.arange(1,10)
day_pred=np.arange(10,20)

In [15]:
len(timeseries_data)

503

In [16]:
len(lst_output)

90

In [17]:
day_new=np.arange(1,504)
day_pred=np.arange(504,594)

In [18]:
from matplotlib.pyplot import figure

In [19]:
figure(figsize=(12, 9), dpi=80)
plt.plot(day_new,timeseries_data)
plt.plot(day_pred,lst_output)